In [30]:
from encoder import QTModel
import torch
import numpy as np
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from nltk.tokenize import sent_tokenize as ST

In [31]:
W2V_PATH = "/home/jingjing/Desktop/InferSent-master/dataset/GloVe/glove.840B.300d.txt"

In [3]:
f = open('/home/jingjing/Desktop/big.txt', 'r')
data = f.read()
splat = data.split("\n\n")

In [4]:
paragraphs = splat

In [5]:
sentences = ['Doctors lead a hard life.',
             'Their life is very busy.',
             'They get up early in the morning and go to the hospital.']
data = []
for p in paragraphs:
    nl = ST(p)
    if len(nl) > 1:
        data.append(ST(p))

In [32]:
f = QTModel()
f.set_w2v_path(W2V_PATH)

In [33]:
def make_target(context_size, dim):
    targets = np.zeros((dim, dim))
    ctxt_sent_pos = list(range(-context_size, context_size+1))
    ctxt_sent_pos.remove(0)
    for ctxt in ctxt_sent_pos:
        targets += np.eye(dim, k=ctxt)
    targets_sum = np.sum(targets,axis=1, keepdims=True)
    targets = targets / targets_sum
    targets = torch.from_numpy(targets)
    return targets

In [34]:
def loss_fn(pred, target):
    mask = 1 - torch.diag(torch.ones(pred.size(1)))
    npred = pred * mask
    s_pred = F.softmax(npred, -1)
    ln = nn.BCELoss(size_average=False)
    losses = ln(s_pred, target)
    return losses

In [35]:
with torch.no_grad():
    f.build_vocab(sentences, True)
    
    embs = f(sentences, 400)
    print(embs)
    targets = make_target(1, len(sentences))
    loss = loss_fn(embs, targets.float())
    print("loss before training: ", loss)

Found 23(/23) words with w2v vectors
Vocab size : 23
Found 23(/23) words with w2v vectors
Vocab size : 23
tensor([[ 13.8343,  13.0828,  12.9733],
        [ 13.0828,  14.3033,  13.3851],
        [ 12.9733,  13.3851,  14.0952]])
loss before training:  tensor(3.7054)


In [36]:
optimizer = optim.Adam(f.parameters(), lr=0.0005)
nn.utils.clip_grad_norm_(f.parameters(), 5)

0.0

In [37]:
print(len(data))

89715


In [55]:
trdata = [[i] for i in range(3)]
for epoch in range(50):
    i = 0
    for instance in data[:3]:
        optimizer.zero_grad()
        #if epoch == 0:
        #    f.build_vocab(instance, True)
        targets = make_target(1, len(instance))
        scores = f(instance, 400)
        
        loss = loss_fn(scores, targets.float())
        #print(loss)
        trdata[i].append(loss.item())
        i += 1
        loss.backward()
        optimizer.step()

In [15]:
with torch.no_grad():
    #f.build_vocab(sentences, True)
    embeddings = f(sentences, 400)
    targets = make_target(1, len(sentences))
    loss = loss_fn(embeddings, targets.float())
    print("loss after training: ", loss)

loss after training:  tensor(19.4918)


In [1]:
import matplotlib.pyplot as plt

for i in range(3):
    plt.figure(i)
    plt.plot(trdata[i][1:])
    plt.show()


NameError: name 'trdata' is not defined

In [15]:
trdata[0]

[0,
 4.1722283363342285,
 1.9811714887619019,
 1.7569588422775269,
 1.5284161567687988,
 1.4439995288848877,
 1.4134989976882935,
 1.4963237047195435,
 1.5102678537368774,
 1.4848254919052124,
 1.5322985649108887]

In [50]:
for name, param in f.named_parameters():
    if param.requires_grad:
        print(name, param.data)

f.enc_gru.weight_ih_l0 tensor([[-3.1235e-03,  1.4690e-02, -2.2814e-02,  ..., -1.8208e-02,
          1.4399e-02, -6.8211e-03],
        [-1.6123e-02, -2.1756e-03, -7.4266e-03,  ...,  3.0341e-02,
          2.2185e-02,  1.9543e-02],
        [-3.0081e-02, -1.0454e-02, -2.6432e-02,  ...,  4.5374e-03,
          1.5086e-02,  1.4460e-02],
        ...,
        [-3.6253e-04, -1.7766e-02,  3.4047e-02,  ...,  1.3577e-02,
          1.6871e-02, -2.6868e-02],
        [-2.5595e-02,  5.4377e-03,  1.3931e-02,  ...,  1.4207e-02,
          2.8216e-02,  9.0383e-03],
        [-2.8259e-02, -1.4220e-02,  2.6841e-02,  ...,  1.2873e-02,
         -7.0035e-04,  3.3505e-03]])
f.enc_gru.weight_hh_l0 tensor([[ 2.1337e-02,  1.5936e-02,  4.5728e-03,  ...,  1.7237e-02,
         -1.1569e-02,  5.0962e-03],
        [ 1.6729e-02, -1.7618e-02, -3.3060e-02,  ..., -2.7745e-02,
          2.7399e-02,  8.0824e-03],
        [ 2.4141e-02, -1.2633e-02, -2.5680e-02,  ...,  5.5447e-03,
          2.1274e-02, -1.1687e-02],
        ...,
